In [1]:
# -*- coding: utf-8 -*-
"""Untitled43.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1hBxgkf6LpRbsZ-P1w3eRwljekhMy_CYW
"""



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from datetime import datetime, timedelta
import io
import base64
import os

# Set matplotlib to use 'Agg' backend to avoid GUI issues
plt.switch_backend('Agg')

# Suppress warnings
warnings.filterwarnings('ignore')

# Dictionary of all NIFTY Bank stocks
BANK_TICKERS = {
    "HDFC Bank": "HDFCBANK.NS",
    "ICICI Bank": "ICICIBANK.NS",
    "State Bank of India": "SBIN.NS",
    "Kotak Mahindra Bank": "KOTAKBANK.NS",
    "Axis Bank": "AXISBANK.NS",
    "IndusInd Bank": "INDUSINDBK.NS",
    "Bandhan Bank": "BANDHANBNK.NS",
    "Federal Bank": "FEDERALBNK.NS",
    "IDFC First Bank": "IDFCFIRSTB.NS",
    "RBL Bank": "RBLBANK.NS",
    "Punjab National Bank": "PNB.NS",
    "Bank of Baroda": "BANKBARODA.NS",
    "Canara Bank": "CANBK.NS",
    "Union Bank of India": "UNIONBANK.NS",
    "Bank of India": "BANKINDIA.NS"
}

# Function to check if a date is a weekend
def is_weekend(date):
    """
    Check if a date is a weekend (Saturday or Sunday).

    Parameters:
    -----------
    date : datetime.date
        Date to check

    Returns:
    --------
    bool
        True if the date is a weekend, False otherwise
    """
    return date.weekday() >= 5  # 5 = Saturday, 6 = Sunday

# Function to load stock data
def load_nifty_bank_data(ticker, start_date, end_date):
    """
    Load historical stock data for a given ticker.

    Parameters:
    -----------
    ticker : str
        Stock ticker symbol (e.g., "HDFCBANK.NS")
    start_date : datetime.date
        Start date for historical data
    end_date : datetime.date
        End date for historical data

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing historical stock data with 'Close' prices and additional features
    """
    try:
        # Convert dates to string format for yfinance
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')

        print(f"Loading data for {ticker} from {start_str} to {end_str}...")

        # Download data from Yahoo Finance
        data = yf.download(ticker, start=start_str, end=end_str)

        if data.empty:
            print(f"No data available for {ticker}")
            return None

        # Keep only close price and create additional features
        df = data[['Close']].copy()

        # Add technical indicators
        # 5-day moving average
        df['MA5'] = df['Close'].rolling(window=5).mean()

        # 20-day moving average
        df['MA20'] = df['Close'].rolling(window=20).mean()

        # Daily returns
        df['Returns'] = df['Close'].pct_change()

        # 20-day volatility
        df['Volatility'] = df['Returns'].rolling(window=20).std()

        # Fill missing values that might be created by rolling windows
        df.fillna(method='bfill', inplace=True)

        print(f"Successfully loaded data with shape {df.shape}")
        return df

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

# Function to create sequences for time series prediction
def create_sequences(data, seq_length):
    """
    Create sequences of data for time series prediction.

    Parameters:
    -----------
    data : numpy.ndarray
        Input data array (scaled feature values)
    seq_length : int
        Sequence length (window size)

    Returns:
    --------
    X : numpy.ndarray
        Input sequences with shape (n_samples, seq_length, n_features)
    y : numpy.ndarray
        Target values with shape (n_samples, 1)
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])  # Target is the Close price

    return np.array(X), np.array(y).reshape(-1, 1)

# Function to preprocess data
def preprocess_data(df, sequence_length=30, test_size=0.2):
    """
    Preprocess stock data for model training.

    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe with stock data
    sequence_length : int
        Length of sequence for time series prediction
    test_size : float
        Proportion of data to use for testing (between 0 and 1)

    Returns:
    --------
    X_train : numpy.ndarray
        Training input sequences
    X_test : numpy.ndarray
        Testing input sequences
    y_train : numpy.ndarray
        Training target values
    y_test : numpy.ndarray
        Testing target values
    scaler : sklearn.preprocessing.MinMaxScaler
        Scaler used for normalization
    train_dates : list
        Dates corresponding to training data
    test_dates : list
        Dates corresponding to testing data
    """
    # Ensure dataframe has data
    if df is None or df.empty or len(df) <= sequence_length:
        print("Insufficient data for preprocessing")
        return None, None, None, None, None, None, None

    try:
        # Extract features
        data = df.values

        # Scale the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data)

        # Create sequences
        X, y = create_sequences(scaled_data, sequence_length)

        if len(X) == 0:
            print("No sequences could be created. Check data length and sequence length.")
            return None, None, None, None, None, None, None

        # Get corresponding dates (offset by sequence_length)
        all_dates = df.index[sequence_length:].tolist()

        # Split into train and test sets
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        train_dates = all_dates[:split_idx]
        test_dates = all_dates[split_idx:]

        return X_train, X_test, y_train, y_test, scaler, train_dates, test_dates

    except Exception as e:
        print(f"Error preprocessing data: {str(e)}")
        return None, None, None, None, None, None, None

# Function to transform scaled predictions back to original scale
def inverse_transform(data, scaler):
    """
    Transform scaled predictions back to original scale.

    Parameters:
    -----------
    data : numpy.ndarray
        Scaled data to transform
    scaler : sklearn.preprocessing.MinMaxScaler
        Scaler used for normalization

    Returns:
    --------
    numpy.ndarray
        Data in original scale
    """
    # Create a copy of the data to avoid modifying the original
    data_copy = data.copy()

    # Prepare dummy array with zeros for all features except the first one (Close price)
    dummy = np.zeros((data_copy.shape[0], scaler.scale_.shape[0]))
    dummy[:, 0] = data_copy.flatten()  # Set the first column to our predictions

    # Inverse transform
    return scaler.inverse_transform(dummy)[:, [0]]  # Return only the Close price column

# Function to get trend indicator and percentage change
def get_trend_indicator(current_value, previous_value):
    """
    Determine trend direction (up or down) and calculate percentage change.

    Parameters:
    -----------
    current_value : float
        Current value
    previous_value : float
        Previous value

    Returns:
    --------
    tuple
        Trend direction ('up', 'down', or 'neutral') and percentage change
    """
    if current_value > previous_value:
        pct_change = ((current_value - previous_value) / previous_value) * 100
        return "up", pct_change
    elif current_value < previous_value:
        pct_change = ((previous_value - current_value) / previous_value) * 100
        return "down", -pct_change
    else:
        return "neutral", 0.0

# Function to format prediction output
def format_prediction_output(date, value, trend=None, pct_change=None):
    """
    Format prediction output with weekend check and trend indicators.

    Parameters:
    -----------
    date : datetime.date
        Date of the prediction
    value : float
        Predicted value
    trend : str
        Trend direction ('up', 'down', or 'neutral')
    pct_change : float
        Percentage change from previous value

    Returns:
    --------
    str
        Formatted output string with price, trend symbol, and percentage change
    """
    if is_weekend(date):
        return "Market is closed on weekends"

    if trend and pct_change is not None:
        trend_symbol = "↑" if trend == "up" else "↓" if trend == "down" else "→"
        return f"₹{value:.2f} {trend_symbol} ({pct_change:.2f}%)"

    return f"₹{value:.2f}"

# Hybrid LSTM+QNN model implementation
class HybridLSTM_QNN:
    """
    A hybrid model combining LSTM (Long Short-Term Memory) and QNN (Quantum Neural Network) architectures
    for time series prediction.

    This model first processes sequential data with an LSTM layer and then passes the output
    to a simulated quantum neural network for further processing.
    """

    def __init__(self, input_shape, hidden_units=64, n_qubits=3, n_layers=2):
        """
        Initialize the hybrid LSTM+QNN model.

        Parameters:
        -----------
        input_shape : tuple
            Shape of input data (sequence_length, n_features)
        hidden_units : int
            Number of hidden units in LSTM layer
        n_qubits : int
            Number of qubits to simulate in QNN
        n_layers : int
            Number of variational layers in QNN
        """
        print(f"Creating Hybrid LSTM+QNN model with {hidden_units} LSTM units and {n_qubits} qubits")
        self.seq_length, self.n_features = input_shape
        self.hidden_units = hidden_units
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # Initialize LSTM parameters
        # Weight matrices
        self.Wf = np.random.randn(hidden_units, hidden_units + self.n_features) * 0.01
        self.Wi = np.random.randn(hidden_units, hidden_units + self.n_features) * 0.01
        self.Wc = np.random.randn(hidden_units, hidden_units + self.n_features) * 0.01
        self.Wo = np.random.randn(hidden_units, hidden_units + self.n_features) * 0.01

        # Bias vectors
        self.bf = np.zeros((hidden_units, 1))
        self.bi = np.zeros((hidden_units, 1))
        self.bc = np.zeros((hidden_units, 1))
        self.bo = np.zeros((hidden_units, 1))

        # Initialize QNN parameters
        # Random parameters for rotation gates
        self.theta_x = np.random.randn(n_qubits, n_layers) * 0.01
        self.theta_y = np.random.randn(n_qubits, n_layers) * 0.01
        self.theta_z = np.random.randn(n_qubits, n_layers) * 0.01

        # Parameters for output layer
        self.Wout = np.random.randn(1, n_qubits) * 0.01
        self.bout = np.zeros((1, 1))

        # Initialize memory for LSTM training
        self.h_cache = {}
        self.c_cache = {}
        self.x_cache = {}
        self.gates_cache = {}

    def sigmoid(self, x):
        """
        Sigmoid activation function.

        Parameters:
        -----------
        x : numpy.ndarray
            Input array

        Returns:
        --------
        numpy.ndarray
            Sigmoid of input
        """
        return 1 / (1 + np.exp(-np.clip(x, -15, 15)))

    def tanh(self, x):
        """
        Tanh activation function.

        Parameters:
        -----------
        x : numpy.ndarray
            Input array

        Returns:
        --------
        numpy.ndarray
            Tanh of input
        """
        return np.tanh(x)

    def lstm_forward(self, x):
        """
        Forward pass through the LSTM part of the network.

        Parameters:
        -----------
        x : numpy.ndarray
            Input sequence with shape (batch_size, seq_length, n_features)

        Returns:
        --------
        numpy.ndarray
            Last hidden state with shape (batch_size, hidden_units)
        """
        batch_size = x.shape[0]

        # Initialize hidden state and cell state
        h_prev = np.zeros((batch_size, self.hidden_units))
        c_prev = np.zeros((batch_size, self.hidden_units))

        # Process each time step
        for t in range(self.seq_length):
            # Current input
            x_t = x[:, t, :]

            # Concatenate input with previous hidden state
            concat = np.concatenate((h_prev, x_t), axis=1)

            # Compute gates
            f_gate = self.sigmoid(np.dot(concat, self.Wf.T) + self.bf.T)
            i_gate = self.sigmoid(np.dot(concat, self.Wi.T) + self.bi.T)
            c_tilde = self.tanh(np.dot(concat, self.Wc.T) + self.bc.T)
            o_gate = self.sigmoid(np.dot(concat, self.Wo.T) + self.bo.T)

            # Update cell state
            c_curr = f_gate * c_prev + i_gate * c_tilde

            # Compute hidden state
            h_curr = o_gate * self.tanh(c_curr)

            # Update for next time step
            h_prev = h_curr
            c_prev = c_curr

        # Return the final hidden state
        return h_prev

    def simulate_rotation(self, state, angle, axis):
        """
        Simulate a quantum rotation gate.

        Parameters:
        -----------
        state : float
            Quantum state value
        angle : float
            Rotation angle
        axis : int
            Rotation axis (0=X, 1=Y, 2=Z)

        Returns:
        --------
        float
            Rotated state value
        """
        if axis == 0:  # RX gate
            return state * np.cos(angle/2) + state * np.sin(angle/2) * 1j
        elif axis == 1:  # RY gate
            return state * np.cos(angle/2) + state * np.sin(angle/2)
        else:  # RZ gate
            return state * np.exp(1j * angle/2)

    def simulate_entanglement(self, states):
        """
        Simulate entanglement between qubits (simplified).

        Parameters:
        -----------
        states : numpy.ndarray
            Array of quantum states

        Returns:
        --------
        numpy.ndarray
            'Entangled' quantum states
        """
        # Simplified entanglement simulation by mixing state values
        n_states = len(states)
        entangled = states.copy()

        for i in range(n_states - 1):
            # Apply a simple operation to simulate entanglement
            entangled[i] = (states[i] + states[i+1]) / np.sqrt(2)
            entangled[i+1] = (states[i] - states[i+1]) / np.sqrt(2)

        return entangled

    def qnn_forward(self, states):
        """
        Forward pass through the QNN part of the network.

        Parameters:
        -----------
        states : numpy.ndarray
            Quantum states

        Returns:
        --------
        numpy.ndarray
            Output of QNN
        """
        batch_size = states.shape[0]

        # Initialize quantum states (simplified representation)
        q_states = np.zeros((batch_size, self.n_qubits), dtype=complex)
        q_states[:, :] = 1.0 + 0.0j  # Start in |0> state (simplified)

        # Encode classical data into quantum states
        for i in range(min(self.n_qubits, states.shape[1])):
            q_states[:, i] = states[:, i]

        # Apply variational quantum layers
        for l in range(self.n_layers):
            # Apply rotation gates
            for q in range(self.n_qubits):
                # X rotation
                q_states[:, q] = self.simulate_rotation(q_states[:, q], self.theta_x[q, l], 0)
                # Y rotation
                q_states[:, q] = self.simulate_rotation(q_states[:, q], self.theta_y[q, l], 1)
                # Z rotation
                q_states[:, q] = self.simulate_rotation(q_states[:, q], self.theta_z[q, l], 2)

            # Apply entanglement between qubits
            for b in range(batch_size):
                q_states[b] = self.simulate_entanglement(q_states[b])

        # Measure qubits (simplified)
        # Take absolute square to get probabilities
        measurements = np.abs(q_states) ** 2

        # Apply output layer
        output = np.dot(measurements, self.Wout.T) + self.bout.T

        return output

    def forward(self, x):
        """
        Forward pass through the hybrid LSTM+QNN model.

        Parameters:
        -----------
        x : numpy.ndarray
            Input data with shape (batch_size, seq_length, n_features)

        Returns:
        --------
        numpy.ndarray
            Predictions with shape (batch_size, 1)
        """
        # Process through LSTM
        lstm_out = self.lstm_forward(x)

        # Process through QNN
        qnn_out = self.qnn_forward(lstm_out)

        return qnn_out

    def train(self, X_train, y_train, epochs=15, batch_size=32, verbose=True):
        """
        Train the hybrid LSTM+QNN model.

        Parameters:
        -----------
        X_train : numpy.ndarray
            Training input data with shape (n_samples, seq_length, n_features)
        y_train : numpy.ndarray
            Training target values with shape (n_samples, 1)
        epochs : int
            Number of training epochs
        batch_size : int
            Batch size for mini-batch training
        verbose : bool
            Whether to print training progress

        Returns:
        --------
        list
            Training loss history
        """
        print("Training Hybrid LSTM+QNN model...")
        n_samples = X_train.shape[0]
        loss_history = []

        # Simple learning rates
        lr_lstm = 0.01
        lr_qnn = 0.05

        for epoch in range(1, epochs + 1):
            # Shuffle data
            indices = np.random.permutation(n_samples)
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]

            epoch_loss = 0

            # Process in batches
            for i in range(0, n_samples, batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                batch_size_actual = X_batch.shape[0]

                # Forward pass
                y_pred = self.forward(X_batch)

                # Compute loss (MSE)
                loss = np.mean((y_pred - y_batch) ** 2)
                epoch_loss += loss * batch_size_actual

                # Compute gradients (simplified)
                # Output layer gradients
                dout = 2 * (y_pred - y_batch) / batch_size_actual
                dWout = np.dot(dout.T, np.abs(np.random.randn(batch_size_actual, self.n_qubits)) ** 2)
                dbout = np.sum(dout, axis=0, keepdims=True).T

                # Simplified update for QNN parameters (random perturbation for demonstration)
                for q in range(self.n_qubits):
                    for l in range(self.n_layers):
                        # Random perturbation as a simple form of gradient
                        self.theta_x[q, l] -= lr_qnn * np.random.randn() * np.mean(dout)
                        self.theta_y[q, l] -= lr_qnn * np.random.randn() * np.mean(dout)
                        self.theta_z[q, l] -= lr_qnn * np.random.randn() * np.mean(dout)

                # Simplified update for LSTM parameters (random perturbation for demonstration)
                self.Wf -= lr_lstm * np.random.randn(*self.Wf.shape) * np.mean(np.abs(dout))
                self.Wi -= lr_lstm * np.random.randn(*self.Wi.shape) * np.mean(np.abs(dout))
                self.Wc -= lr_lstm * np.random.randn(*self.Wc.shape) * np.mean(np.abs(dout))
                self.Wo -= lr_lstm * np.random.randn(*self.Wo.shape) * np.mean(np.abs(dout))

                self.bf -= lr_lstm * np.random.randn(*self.bf.shape) * np.mean(np.abs(dout))
                self.bi -= lr_lstm * np.random.randn(*self.bi.shape) * np.mean(np.abs(dout))
                self.bc -= lr_lstm * np.random.randn(*self.bc.shape) * np.mean(np.abs(dout))
                self.bo -= lr_lstm * np.random.randn(*self.bo.shape) * np.mean(np.abs(dout))

                # Update output layer
                self.Wout -= lr_lstm * dWout
                self.bout -= lr_lstm * dbout

            # Calculate average epoch loss
            epoch_loss /= n_samples
            loss_history.append(epoch_loss)

            # Print progress
            if verbose and (epoch == 1 or epoch % 5 == 0 or epoch == epochs):
                print(f"Epoch {epoch}/{epochs}, Loss: {epoch_loss:.6f}")

        print("Hybrid LSTM+QNN model training completed")
        return loss_history

    def predict(self, X):
        """
        Make predictions using the trained hybrid model.

        Parameters:
        -----------
        X : numpy.ndarray
            Input data with shape (n_samples, seq_length, n_features)

        Returns:
        --------
        numpy.ndarray
            Predictions with shape (n_samples, 1)
        """
        return self.forward(X)

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    """
    Calculate evaluation metrics for model performance.

    Parameters:
    -----------
    y_true : numpy.ndarray
        True values
    y_pred : numpy.ndarray
        Predicted values

    Returns:
    --------
    dict
        Dictionary containing metrics:
        - MSE (Mean Squared Error)
        - RMSE (Root Mean Squared Error)
        - MAE (Mean Absolute Error)
        - MAPE (Mean Absolute Percentage Error)
        - R² (R-squared)
        - Directional Accuracy (percentage of correct trend predictions)
        - Forecast Accuracy (100 - MAPE)
    """
    # Ensure inputs are 1D arrays
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    # Calculate standard metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)

    # Calculate MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Calculate R-squared
    r2 = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

    # Calculate directional accuracy (percentage of correct trend predictions)
    true_diff = np.diff(y_true)
    pred_diff = np.diff(y_pred)
    correct_direction = (true_diff * pred_diff) > 0  # Both have same sign
    directional_accuracy = np.mean(correct_direction) * 100

    # Calculate forecast accuracy (how close predictions are to actual values)
    forecast_accuracy = 100 - mape

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape,
        'R²': r2,
        'Directional Accuracy (%)': directional_accuracy,
        'Forecast Accuracy (%)': forecast_accuracy
    }

# Function to display metrics
def display_metrics(metrics, bank_name):
    """
    Display metrics in a formatted way.

    Parameters:
    -----------
    metrics : dict
        Dictionary of metrics from calculate_metrics()
    bank_name : str
        Name of the bank for display purposes
    """
    print(f"Model Metrics for {bank_name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

# Function to plot stock predictions
def plot_stock_predictions(df, predictions_data, bank_name):
    """
    Create a plot showing historical data and predictions with blue dots for current day
    and red dots for next day predictions.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing stock data
    predictions_data : dict
        Dictionary containing prediction data:
        - test_dates: dates for test predictions
        - test_preds: historical predictions
        - current_date: date for current day prediction
        - current_pred: current day prediction
        - next_date: date for next day prediction
        - next_pred: next day prediction
        - previous_value: previous day actual value
    bank_name : str
        Name of the bank for display

    Returns:
    --------
    str
        Filename of the saved plot
    """
    try:
        plt.figure(figsize=(12, 6))

        # Plot historical data as a gray line
        plt.plot(df.index, df['Close'], label='Historical Close Price', color='gray', alpha=0.7)

        # Plot historical predictions as a green line
        if 'test_dates' in predictions_data and 'test_preds' in predictions_data and len(predictions_data['test_preds']) > 0:
            plt.plot(predictions_data['test_dates'], predictions_data['test_preds'],
                     label='Historical Predictions', color='green', alpha=0.8)

        # Add current day prediction as a BLUE DOT
        if 'current_date' in predictions_data and 'current_pred' in predictions_data:
            # Plot blue dot for current day prediction
            plt.scatter([predictions_data['current_date']], [predictions_data['current_pred']],
                        color='blue', label='Current Day Prediction', s=150, zorder=5)

            # Add label with price
            plt.annotate(f"₹{predictions_data['current_pred']:.2f}",
                         (predictions_data['current_date'], predictions_data['current_pred']),
                         textcoords="offset points", xytext=(0,10), ha='center')

            # Add trend arrow
            if 'previous_value' in predictions_data:
                trend, pct = get_trend_indicator(predictions_data['current_pred'],
                                                 predictions_data['previous_value'])
                if trend == 'up':
                    plt.annotate('↑', (predictions_data['current_date'], predictions_data['current_pred']),
                                 textcoords="offset points", xytext=(7,-5), color='green', fontsize=15)
                elif trend == 'down':
                    plt.annotate('↓', (predictions_data['current_date'], predictions_data['current_pred']),
                                 textcoords="offset points", xytext=(7,-5), color='red', fontsize=15)

        # For next day prediction
        if 'next_date' in predictions_data and 'next_pred' in predictions_data:
            if is_weekend(predictions_data['next_date']):
                # For weekend predictions, show the last closing price (previous value)
                if 'previous_value' in predictions_data:
                    # Use a gray dot for weekend (market closed)
                    plt.scatter([predictions_data['next_date']], [predictions_data['previous_value']],
                              color='gray', label='Weekend (Market Closed)', s=150, zorder=5)

                    # Add "Market Closed" label
                    plt.annotate("Market Closed (Weekend)",
                                (predictions_data['next_date'], predictions_data['previous_value']),
                                textcoords="offset points", xytext=(0,10), ha='center')

                    # Highlight weekend with shaded area
                    plt.axvspan(predictions_data['next_date'] - timedelta(hours=12),
                                predictions_data['next_date'] + timedelta(hours=12),
                                alpha=0.2, color='gray')
            else:
                # For weekdays, plot the regular red dot for next day prediction
                plt.scatter([predictions_data['next_date']], [predictions_data['next_pred']],
                            color='red', label='Next Day Prediction', s=150, zorder=5)

                # Add label with price
                plt.annotate(f"₹{predictions_data['next_pred']:.2f}",
                             (predictions_data['next_date'], predictions_data['next_pred']),
                             textcoords="offset points", xytext=(0,10), ha='center')

                # Add trend arrow
                if 'current_pred' in predictions_data:
                    trend, pct = get_trend_indicator(predictions_data['next_pred'],
                                                     predictions_data['current_pred'])
                    if trend == 'up':
                        plt.annotate('↑', (predictions_data['next_date'], predictions_data['next_pred']),
                                     textcoords="offset points", xytext=(7,-5), color='green', fontsize=15)
                    elif trend == 'down':
                        plt.annotate('↓', (predictions_data['next_date'], predictions_data['next_pred']),
                                     textcoords="offset points", xytext=(7,-5), color='red', fontsize=15)

        # Set title and labels
        plt.title(f'{bank_name} Stock Price Prediction (Hybrid LSTM+QNN)', fontsize=14)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Price (₹)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='best')

        # Format x-axis dates
        plt.gcf().autofmt_xdate()
        plt.tight_layout()

        # Save the plot to a file
        filename = f'{bank_name.replace(" ", "_").lower()}_hybrid_predictions.png'
        plt.savefig(filename, dpi=100, bbox_inches='tight')
        print(f"Plot saved to {filename}")

        # Check if the file was created
        if os.path.exists(filename):
            print(f"Verified: File {filename} was successfully created")
        else:
            print(f"Warning: File {filename} was not created")

        # Show the plot image in the output
        # Read the file and encode it in base64
        with open(filename, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

        # Print the image path for reference
        print(f"Successfully created visualization for {bank_name} in file {filename}")

        plt.close()  # Close the figure to free memory

        return filename

    except Exception as e:
        print(f"Error plotting predictions: {str(e)}")
        return None

# Function to display prediction details
def display_prediction_details(bank_name, predictions_data, dates):
    """
    Display detailed prediction information for a specific bank.

    Parameters:
    -----------
    bank_name : str
        Name of the bank
    predictions_data : dict
        Dictionary containing prediction data
    dates : tuple
        Tuple containing (previous_date, current_date, next_date)
    """
    prev_date, current_date, next_date = dates
    previous_day_actual = predictions_data.get('previous_value')
    current_day_pred = predictions_data.get('current_pred')
    next_day_pred = predictions_data.get('next_pred')

    print(f"\n{'='*30} {bank_name} Price Predictions {'='*30}")

    # Previous day (actual closing price)
    prev_day_output = format_prediction_output(
        prev_date,
        previous_day_actual
    ) if previous_day_actual is not None else "N/A"
    print(f"Previous Close ({prev_date.strftime('%Y-%m-%d')}): {prev_day_output}")

    # Current day prediction
    if current_day_pred is not None and previous_day_actual is not None:
        trend, pct_change = get_trend_indicator(current_day_pred, previous_day_actual)
        current_day_output = format_prediction_output(
            current_date,
            current_day_pred,
            trend,
            pct_change
        )
        print(f"Current Day ({current_date.strftime('%Y-%m-%d')}): {current_day_output}")
    else:
        print(f"Current Day ({current_date.strftime('%Y-%m-%d')}): N/A")

    # Next day prediction
    if is_weekend(next_date):
        print(f"Next Day ({next_date.strftime('%Y-%m-%d')}): Market is closed on weekends")
    elif next_day_pred is not None and current_day_pred is not None:
        trend, pct_change = get_trend_indicator(next_day_pred, current_day_pred)
        next_day_output = format_prediction_output(
            next_date,
            next_day_pred,
            trend,
            pct_change
        )
        print(f"Next Day ({next_date.strftime('%Y-%m-%d')}): {next_day_output}")
    else:
        print(f"Next Day ({next_date.strftime('%Y-%m-%d')}): N/A")

# Function to display prediction summary
def display_prediction_summary(results):
    """
    Display a summary of predictions for all banks.

    Parameters:
    -----------
    results : dict
        Dictionary of results for each bank
    """
    print("\n" + "="*30 + " Summary of Price Predictions " + "="*30)

    for bank_name, result in results.items():
        print(f"\n{bank_name}:")

        # Previous day
        if result['previous_day'] is not None:
            print(f"  Previous Close: ₹{result['previous_day']:.2f}")
        else:
            print("  Previous Close: N/A")

        # Current day
        if result['current_day'] is not None and result['previous_day'] is not None:
            trend, pct_change = get_trend_indicator(result['current_day'], result['previous_day'])
            trend_symbol = "↑" if trend == "up" else "↓" if trend == "down" else "→"
            print(f"  Current Day: ₹{result['current_day']:.2f} {trend_symbol} ({pct_change:.2f}%)")
        else:
            print("  Current Day: N/A")

        # Next day
        if 'next_date' in result and is_weekend(result['next_date']):
            print(f"  Next Day: Market is closed on weekends")
        elif result['next_day'] is not None and result['current_day'] is not None:
            trend, pct_change = get_trend_indicator(result['next_day'], result['current_day'])
            trend_symbol = "↑" if trend == "up" else "↓" if trend == "down" else "→"
            print(f"  Next Day: ₹{result['next_day']:.2f} {trend_symbol} ({pct_change:.2f}%)")
        else:
            print("  Next Day: N/A")

        # Display accuracy metrics
        if 'metrics' in result and result['metrics']:
            print(f"  Forecast Accuracy: {result['metrics'].get('Forecast Accuracy (%)', 'N/A'):.2f}%")
            print(f"  Directional Accuracy: {result['metrics'].get('Directional Accuracy (%)', 'N/A'):.2f}%")

# Main function
def main():
    """
    Main function that orchestrates the entire prediction process for all 15 NIFTY Bank stocks.
    """
    # Date range for data
    # For demonstration purposes, use dates that specifically include a weekend for testing
    # Use Thursday as current day, so next day will be Friday but the day after will be a weekend
    end_date = datetime(2025, 4, 24).date()  # Thursday
    start_date = end_date - timedelta(days=365*2)  # 2 years of data

    # Model parameters
    sequence_length = 30  # Length of input sequences

    # Store results for summary
    results = {}

    try:
        # Process all 15 NIFTY Bank stocks
        banks_to_process = list(BANK_TICKERS.keys())

        for bank_name in banks_to_process:
            ticker = BANK_TICKERS[bank_name]
            print(f"\n{'='*60}")
            print(f"Processing {bank_name} (Ticker: {ticker})")
            print(f"{'='*60}")

            # Step 1: Load data
            df = load_nifty_bank_data(ticker, start_date, end_date)

            if df is not None and not df.empty:
                # Step 2: Preprocess data
                print("Preprocessing data...")
                X_train, X_test, y_train, y_test, scaler, train_dates, test_dates = preprocess_data(
                    df, sequence_length=sequence_length, test_size=0.2
                )

                if X_train is not None and y_train is not None:
                    # Step 3: Create and train the hybrid model
                    print("Creating and training hybrid LSTM+QNN model...")
                    model = HybridLSTM_QNN(
                        input_shape=(X_train.shape[1], X_train.shape[2]),
                        hidden_units=64,
                        n_qubits=3,
                        n_layers=2
                    )

                    history = model.train(X_train, y_train, epochs=15, batch_size=32)

                    # Step 4: Make predictions
                    print("Generating predictions...")
                    train_pred = model.predict(X_train)
                    test_pred = model.predict(X_test) if X_test is not None and len(X_test) > 0 else np.array([])

                    # Get the latest sequence for next day prediction
                    latest_sequence = X_test[-1:] if X_test is not None and len(X_test) > 0 else X_train[-1:]

                    # Predict next day
                    next_day_pred = model.predict(latest_sequence)[0][0]
                    next_day_pred = inverse_transform(np.array([[next_day_pred]]), scaler)[0][0]

                    # Get current day prediction (last day in test set)
                    current_day_pred = inverse_transform(test_pred[-1:], scaler)[0][0] if len(test_pred) > 0 else None

                    # Get previous day actual (last day in test set)
                    previous_day_actual = inverse_transform(y_test[-1:], scaler)[0][0] if y_test is not None and len(y_test) > 0 else None

                    # Step 5: Calculate metrics
                    metrics = calculate_metrics(
                        inverse_transform(y_test, scaler),
                        inverse_transform(test_pred, scaler)
                    ) if len(test_pred) > 0 and len(y_test) > 0 else {}

                    # Display metrics
                    if metrics:
                        display_metrics(metrics, bank_name)

                    # Use consistent dates for visualization
                    prev_date = end_date - timedelta(days=1)  # April 23, 2025 (Wednesday)
                    current_date = end_date  # Thursday, April 24, 2025
                    next_date = current_date + timedelta(days=1)  # Friday, April 25, 2025

                    # Check if next day is a weekend
                    weekend_next = is_weekend(next_date)

                    # Setup date for the day after next (to test weekend handling)
                    day_after_next = next_date + timedelta(days=1)  # Should be Saturday, April 26, 2025
                    weekend_day_after = is_weekend(day_after_next)

                    if weekend_day_after:
                        print(f"Checking weekend handling: {day_after_next.strftime('%Y-%m-%d')} is a weekend - will display 'Market is closed'")

                    # If weekend, use the previous value (no change) for visualization
                    if weekend_next:
                        next_day_pred_display = previous_day_actual
                        print(f"Next day {next_date.strftime('%Y-%m-%d')} is a weekend - market will be closed")
                    else:
                        next_day_pred_display = next_day_pred

                    # Store results for summary
                    results[bank_name] = {
                        'previous_day': previous_day_actual,
                        'current_day': current_day_pred,
                        'next_day': next_day_pred_display,
                        'next_date': next_date,  # Include the next date to check if it's a weekend
                        'metrics': metrics
                    }

                    # Step 6: Create prediction data for plotting
                    predictions_data = {
                        'test_dates': test_dates,
                        'test_preds': inverse_transform(test_pred, scaler).flatten() if len(test_pred) > 0 else [],
                        'current_date': current_date,
                        'current_pred': current_day_pred,
                        'next_date': next_date,
                        'next_pred': next_day_pred_display,
                        'previous_value': previous_day_actual,
                        'is_weekend_next': weekend_next
                    }

                    # Step 7: Generate visualization plot with blue dot for current day and red dot for next day
                    plot_file = plot_stock_predictions(df, predictions_data, bank_name)

                    if plot_file:
                        print(f"Successfully created visualization for {bank_name} in file {plot_file}")
                    else:
                        print(f"Failed to create visualization for {bank_name}")

                    # Step 8: Display predictions
                    display_prediction_details(
                        bank_name,
                        predictions_data,
                        (prev_date, current_date, next_date)
                    )

                    # Special test for weekend on day after next
                    if weekend_day_after:
                        print(f"Testing weekend handling: {day_after_next.strftime('%Y-%m-%d')}: Market is closed on weekends")
                else:
                    print(f"Error: Unable to preprocess data for {bank_name}")
            else:
                print(f"Error: Unable to load data for {bank_name}")

        # Step 9: Display summary of all predictions
        print("\n" + "="*30 + " SUMMARY OF PREDICTIONS " + "="*30)
        display_prediction_summary(results)

        # Final check for weekend handling
        weekend_dates = [datetime(2025, 4, 26).date(), datetime(2025, 4, 27).date()]  # Saturday and Sunday
        for date in weekend_dates:
            is_weekend_check = is_weekend(date)
            if is_weekend_check:
                print(f"Weekend Check: {date.strftime('%Y-%m-%d')} is a weekend - Market is closed on weekends")

    except Exception as e:
        print(f"An error occurred in the main function: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


Processing HDFC Bank (Ticker: HDFCBANK.NS)
Loading data for HDFCBANK.NS from 2023-04-25 to 2025-04-24...


[*********************100%***********************]  1 of 1 completed


Successfully loaded data with shape (492, 5)
Preprocessing data...
Creating and training hybrid LSTM+QNN model...
Creating Hybrid LSTM+QNN model with 64 LSTM units and 3 qubits
Training Hybrid LSTM+QNN model...
Epoch 1/15, Loss: 0.139087
Epoch 5/15, Loss: 0.047020
Epoch 10/15, Loss: 0.032938
Epoch 15/15, Loss: 0.031664
Hybrid LSTM+QNN model training completed
Generating predictions...
Model Metrics for HDFC Bank:
MSE: 8680.40
RMSE: 93.17
MAE: 85.16
MAPE (%): 9.71
R²: -5.08
Directional Accuracy (%): 47.83
Forecast Accuracy (%): 90.29
Checking weekend handling: 2025-04-26 is a weekend - will display 'Market is closed'
Plot saved to hdfc_bank_hybrid_predictions.png
Verified: File hdfc_bank_hybrid_predictions.png was successfully created
Successfully created visualization for HDFC Bank in file hdfc_bank_hybrid_predictions.png
Successfully created visualization for HDFC Bank in file hdfc_bank_hybrid_predictions.png

============================== HDFC Bank Price Predictions ================

[*********************100%***********************]  1 of 1 completed


Successfully loaded data with shape (492, 5)
Preprocessing data...
Creating and training hybrid LSTM+QNN model...
Creating Hybrid LSTM+QNN model with 64 LSTM units and 3 qubits
Training Hybrid LSTM+QNN model...
Epoch 1/15, Loss: 0.151762
Epoch 5/15, Loss: 0.072401
Epoch 10/15, Loss: 0.060497
Epoch 15/15, Loss: 0.059365
Hybrid LSTM+QNN model training completed
Generating predictions...
Model Metrics for ICICI Bank:
MSE: 46150.88
RMSE: 214.83
MAE: 208.56
MAPE (%): 16.29
R²: -16.39
Directional Accuracy (%): 52.17
Forecast Accuracy (%): 83.71
Checking weekend handling: 2025-04-26 is a weekend - will display 'Market is closed'
Plot saved to icici_bank_hybrid_predictions.png
Verified: File icici_bank_hybrid_predictions.png was successfully created
Successfully created visualization for ICICI Bank in file icici_bank_hybrid_predictions.png
Successfully created visualization for ICICI Bank in file icici_bank_hybrid_predictions.png

============================== ICICI Bank Price Predictions ===

...

Successfully loaded data with shape (492, 5)
Preprocessing data...
Creating and training hybrid LSTM+QNN model...
Creating Hybrid LSTM+QNN model with 64 LSTM units and 3 qubits
Training Hybrid LSTM+QNN model...
Epoch 1/15, Loss: 0.337225
Epoch 5/15, Loss: 0.121298
Epoch 10/15, Loss: 0.088344
Epoch 15/15, Loss: 0.085416
Hybrid LSTM+QNN model training completed
Generating predictions...
Model Metrics for Canara Bank:
MSE: 51.31
RMSE: 7.16
MAE: 5.95
MAPE (%): 6.58
R²: -0.00
Directional Accuracy (%): 45.65
Forecast Accuracy (%): 93.42
Checking weekend handling: 2025-04-26 is a weekend - will display 'Market is closed'
Plot saved to canara_bank_hybrid_predictions.png
Verified: File canara_bank_hybrid_predictions.png was successfully created
Successfully created visualization for Canara Bank in file canara_bank_hybrid_predictions.png
Successfully created visualization for Canara Bank in file canara_bank_hybrid_predictions.png

============================== Canara Bank Price Predictions ====

[*********************100%***********************]  1 of 1 completed


Successfully loaded data with shape (492, 5)
Preprocessing data...
Creating and training hybrid LSTM+QNN model...
Creating Hybrid LSTM+QNN model with 64 LSTM units and 3 qubits
Training Hybrid LSTM+QNN model...
Epoch 1/15, Loss: 0.275690
Epoch 5/15, Loss: 0.086196
Epoch 10/15, Loss: 0.056876
Epoch 15/15, Loss: 0.054328
Hybrid LSTM+QNN model training completed
Generating predictions...
Model Metrics for Union Bank of India:
MSE: 47.89
RMSE: 6.92
MAE: 5.66
MAPE (%): 4.95
R²: -0.03
Directional Accuracy (%): 43.48
Forecast Accuracy (%): 95.05
Checking weekend handling: 2025-04-26 is a weekend - will display 'Market is closed'
Plot saved to union_bank_of_india_hybrid_predictions.png
Verified: File union_bank_of_india_hybrid_predictions.png was successfully created
Successfully created visualization for Union Bank of India in file union_bank_of_india_hybrid_predictions.png
Successfully created visualization for Union Bank of India in file union_bank_of_india_hybrid_predictions.png

=========

[*********************100%***********************]  1 of 1 completed


Successfully loaded data with shape (492, 5)
Preprocessing data...
Creating and training hybrid LSTM+QNN model...
Creating Hybrid LSTM+QNN model with 64 LSTM units and 3 qubits
Training Hybrid LSTM+QNN model...
Epoch 1/15, Loss: 0.262775
Epoch 5/15, Loss: 0.083457
Epoch 10/15, Loss: 0.056254
Epoch 15/15, Loss: 0.053755
Hybrid LSTM+QNN model training completed
Generating predictions...
Model Metrics for Bank of India:
MSE: 76.08
RMSE: 8.72
MAE: 7.46
MAPE (%): 7.68
R²: -0.75
Directional Accuracy (%): 57.61
Forecast Accuracy (%): 92.32
Checking weekend handling: 2025-04-26 is a weekend - will display 'Market is closed'
Plot saved to bank_of_india_hybrid_predictions.png
Verified: File bank_of_india_hybrid_predictions.png was successfully created
Successfully created visualization for Bank of India in file bank_of_india_hybrid_predictions.png
Successfully created visualization for Bank of India in file bank_of_india_hybrid_predictions.png

============================== Bank of India Price 